In [1]:
import os

### Loading tavily stuffs
from langchain.tools.tavily_search import TavilySearchResults 

### Loading langchain stuffs
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

### some more shitty stuffs
import requests
from bs4 import BeautifulSoup
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

import warnings
warnings.filterwarnings('ignore') # avoid warning messages importing packages

import json

REQUEST_PER_QUESTION = 2

In [2]:
from dotenv import load_dotenv
load_dotenv()

# Tavily API Key
auth_key_tavily = os.environ.get("TAVILY_API_KEY")

# HuggingFace API Key
auth_key_hf = os.environ.get("HF_KEY")

# Langchain key
langchain_key = os.environ.get("LANGCHAIN_API_KEY")

In [3]:
from huggingface_hub import snapshot_download
from pathlib import Path

# snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir='./Mistral_7B_Instruct_v0.3/')

In [4]:
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, AssistantMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

import torch
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig, pipeline

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

mistral_models_path = './Mistral_7B_Instruct_v0.3/' 

tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
# model = Transformer.from_file({mistral_models_path})
model = AutoModelForCausalLM.from_pretrained('mistralai/Mistral-7B-Instruct-v0.3', quantization_config=quantization_config)

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])
tokens = tokenizer.encode_chat_completion(completion_request).tokens
generated_ids = model.generate(torch.tensor([tokens]).to('cuda'), max_new_tokens=128)
result = tokenizer.decode(generated_ids[0].tolist())
print(result)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Explain Machine Learning to me in a nutshell. Machine Learning (ML) is a subset of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed.

In simpler terms, it's like teaching a computer to recognize patterns and make decisions based on that learning. Here's a breakdown:

1. Data Collection: Gather a large amount of data related to the problem you want to solve. For example, if you want to predict house prices, you'd need data about houses and their prices.

2. Data Preparation: Clean and preprocess the data to make it suitable for learning


In [5]:
# messages = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]
# 
# encodeds = tokenizer_auto.apply_chat_template(messages, return_tensors="pt")
# 
# generated_ids = model.generate(encodeds.to('cuda'), max_new_tokens=64, do_sample=True)
# decoded = tokenizer_auto.batch_decode(generated_ids)
# print(decoded[0])

In [6]:
from langchain_huggingface import (
    HuggingFacePipeline, HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
)

from transformers import AutoTokenizer

tokenizer_auto = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.3', max_length=128, skip_special_tokens=True)
tokenizer_auto.pad_token = tokenizer_auto.eos_token

pipe = pipeline(
    task="text-generation", 
    model=model, 
    tokenizer=tokenizer_auto, 
    do_sample=True, 
    max_new_tokens=128,
    truncation='only_first',
)

llm = HuggingFacePipeline(
    pipeline=pipe,
)

# llm = HuggingFaceEndpoint(
#     endpoint_url = "https://api-inference.huggingface.co/models/google/gemma-2b-it",
#     task='text-generation',
#     max_new_tokens=64,
#     top_k=10,
#     top_p=0.9,
#     repetition_penalty=1.03,
#     timeout=240,
#     huggingfacehub_api_token=auth_key_hf
# )

In [7]:
llm('<s><INST>Your task is to make a ordered list of the items</INST> for items=(banana, apple), your output: 1. apple, 2. banana.</s><INST> Make list for items "eggs, bacon, toast"</INST>')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'<s><INST>Your task is to make a ordered list of the items</INST> for items=(banana, apple), your output: 1. apple, 2. banana.</s><INST> Make list for items "eggs, bacon, toast"</INST> 1. eggs\n\n2. bacon\n3. toast\n\nI add numbers and a comma after each item.'

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [9]:
from duckduckgo_search import DDGS

def web_search(query:str, num_results:int=2):
    results = DDGS().text(query, max_results=num_results)
    return [r["href"] for r in results] 

def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the webpage using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            # Extract the text from the webpage
            text = soup.get_text(separator=' ', strip=True)
            return text
        else:
            return f"Failed to retrieve the webpage. \
                    Status code: {response.status_code}"
    except Exception as e:
        return f"An error occurred: {e}"

In [10]:
web_search("What is the impact of machine learning on audio faking?")

['https://www.kpbs.org/news/science-technology/2024/04/05/using-ai-to-detect-ai-generated-deepfakes-can-work-for-audio-but-not-always',
 'https://news.mit.edu/2024/what-you-need-to-know-audio-deepfakes-0315']

In [11]:
# summary_template = f"use the {text} to summarize the question: {question}"

# def summary_template(question:str, text:str):
#     return f"""<s><INST>Answer the question:"{question}" by summarizing the text: "{text}" if the question cannot be answered using the text.</INST>"""

# summary_template = """<s><INST>Answer the question:"{question}" by summarizing the text: "{text}" if the question cannot be answered using the text.</INST>"""
# SUMMARY_PROMPT = ChatPromptTemplate.from_template(summary_template)

SUMMARY_PROMPT = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful AI bot.'),
    ('user', 'Answer the question:"{question}" by summarizing the text: "{text}". If the question cannot be answered using the text, return the text.' )
])

scrape_and_summarize_chain = RunnablePassthrough.assign(
    summary=RunnablePassthrough.assign(
        text=lambda x: scrape_text(x["url"])[:10]
    ) | SUMMARY_PROMPT | llm | StrOutputParser()
) | (lambda x: f"URL: {x['url']}\n\nSUMMARY: {x['summary']}")

In [12]:
web_search_chain = (RunnablePassthrough.assign(
    urls = lambda x: web_search(x["question"])
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) 
  | scrape_and_summarize_chain.map())

In [13]:
web_search_chain.invoke({"question": "What is the impact of machine learning on audio faking?"})

['URL: https://news.mit.edu/2024/what-you-need-to-know-audio-deepfakes-0315\n\nSUMMARY: System: You are a helpful AI bot.\nHuman: Answer the question:"What is the impact of machine learning on audio faking?" by summarizing the text: "3 Question". If the question cannot be answered using the text, return the text.\n\nAnswer: "Machine learning significantly impacts audio faking, as it enables the creation of more realistic synthetic audio waveforms. This is demonstrated especially in deep learning models such as WaveNet and Tacotron, which leverage auto-regressive sequence-to-sequence training for more natural sounding voices and even mimicking an individual\'s voice. Deepfakes, a form of audio fake, have gained notoriety with the application of machine learning to manipulate audio content."',
 'URL: https://www.kpbs.org/news/science-technology/2024/04/05/using-ai-to-detect-ai-generated-deepfakes-can-work-for-audio-but-not-always\n\nSUMMARY: System: You are a helpful AI bot.\nHuman: Answ

In [14]:
import regex as re

In [15]:
search_template = """
        <s>
        <INST>
            Write only 2 google search queries to search from the following question: "{question}". 
            Return a numbered list of the queries.
        </INST>
    """

SEARCH_PROMPT = ChatPromptTemplate.from_messages([
    ('user', 'Write only 2 google search queries to search from the following question: "{question}". Return a numbered list of the queries.' ),   
])

In [16]:
search_question_chain = (SEARCH_PROMPT | llm | StrOutputParser() | (lambda x: re.findall(r'[1-9]\.\s?"(.*)"\n?', x)))

In [17]:
search_qs_test = search_question_chain.invoke({'question': 'What is the impact of machine learning on audio faking?'})

In [18]:
search_qs_test

['Impact of machine learning on audio faking',
 'Machine learning and audio fake news']

In [19]:
full_research_chain = search_question_chain |  (lambda x: [{"question": q} for q in x]) | web_search_chain.map()

In [20]:
full_research_chain.invoke({'question': 'What is the impact of machine learning on audio faking?'})

[['URL: https://news.mit.edu/2024/what-you-need-to-know-audio-deepfakes-0315\n\nSUMMARY: System: You are a helpful AI bot.\nHuman: Answer the question:"Impact of Machine Learning on Audio Faking" by summarizing the text: "3 Question". If the question cannot be answered using the text, return the text.\n\nAssistant: The text "3 Question" does not contain sufficient information to provide an answer about the impact of Machine Learning on Audio Faking. To clearly understand the impact, it would be helpful to have more specific information or questions, such as "How has Machine Learning influenced the creation of fake audio?", "What are some examples of audio faking using Machine Learning?", or "What challenges and solutions are there when it comes to detecting fake audio using Machine Learning?". I hope this helps!',
  'URL: https://huggingface.co/blog/Andyrasika/deepfake-detect\n\nSUMMARY: System: You are a helpful AI bot.\nHuman: Answer the question:"Impact of Machine Learning on Audio 

In [21]:
import gc
torch.cuda.empty_cache()
gc.collect()

21384

In [22]:
WRITER_SYSTEM_PROMPT = """
        <s>
        <INST>
            You are an AI critical thinker research assistant. 
            Your sole purpose is to write 
            well written, critically acclaimed, 
            objective and structured reports on given text or task. 
        </INST>
    """

# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """<s><INST>
    Using the information "{research_summary}", answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career.</INST>"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
)

In [23]:
def collapse_list_of_lists(list_of_lists):
    content = []
    for l in list_of_lists:
        content.append("\n\n".join(l))
    return "\n\n".join(content)

In [24]:
chain = (RunnablePassthrough.assign(
    research_summary=full_research_chain | collapse_list_of_lists
) | prompt | llm | StrOutputParser())

In [25]:
chain.invoke({"question":'what is the impact of machine learning on audio faking'})

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


'System: \n        <s>\n        <INST>\n            You are an AI critical thinker research assistant. \n            Your sole purpose is to write \n            well written, critically acclaimed, \n            objective and structured reports on given text or task. \n        </INST>\n    \nHuman: <s><INST>\n    Using the information "URL: https://news.mit.edu/2024/what-you-need-to-know-audio-deepfakes-0315\n\nSUMMARY: System: You are a helpful AI bot.\nHuman: Answer the question:"Impact of machine learning on audio faking" by summarizing the text: "3 Question". If the question cannot be answered using the text, return the text.\nAI System: The text "3 Question" does not directly answer the question "Impact of machine learning on audio faking". The 3 Question text appears to be a reference to a specific conversation or document, and does not contain information about machine learning\'s impact on audio faking. The impact of machine learning on audio faking is significant, as it has ena

In [26]:
# # set up the agent
# llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)
# search = TavilySearchAPIWrapper()
# tavily_tool = TavilySearchResults(api_wrapper=search)
# 
# # initialize the agent
# agent_chain = initialize_agent(
#     [tavily_tool],
#     llm,
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
# )
# 
# # run the agent
# agent_chain.run(
#     "What happened in the latest burning man floods?",
# )

In [27]:
search = TavilySearchResults(max_results=2)
search.invoke('look for leclerc')

[{'url': 'https://www.the-race.com/formula-1/charles-leclerc-new-ferrari-contract/',
  'content': 'Charles Leclerc will stay with Ferrari\'s Formula 1 team for "several more seasons" after signing a new long-term contract. At the end of last year, Leclerc was rumoured to have been close to inking a mega five-year deal with Ferrari that would have kept him there until 2029. Now Ferrari has confirmed it will retain Leclerc for several years beyond the end of his current contract which expires ...'},
 {'url': 'https://www.si.com/fannation/racing/f1briefings/news/f1-news-charles-leclerc-gives-emotional-speech-after-first-monaco-win-01hytrpzqtpw',
  'content': "He fell in love with F1 at the young age of 7 after hearing the scream of naturally aspirated V10s echo through his grandparents' lounge. That year he watched as Michael Schumacher took home his ..."}]

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.retrievers import TavilySearchAPIRetriever


retriever = TavilySearchAPIRetriever(k=2)
chat_template = """Answer the question based only on the context provided.
Context: {context}

Question: {question}"""

prompt = ChatPromptTemplate.from_template(chat_template)
chain = (
        RunnablePassthrough.assign(context=(lambda x: x["question"]) | search)
        | prompt
        | llm 
        | StrOutputParser()
)

In [29]:
chain.invoke({"question": "how many units did breath of the wild sell in 2020?"})

'Human: Answer the question based only on the context provided.\nContext: [{\'url\': \'https://www.reddit.com/r/zelda/comments/k2iiop/botw_breath_of_the_wild_has_sold_34m_copies/\', \'content\': \'Breath of the Wild was a launch title for the Switch it was the most popular title to buy with a brand new Switch back in 2017. All of the games that have sold more copies than Breath of the Wild have multiplayer features where you can play with your buddies. Breath of the Wild is a single player experience it can provide with many hours worth ...\'}, {\'url\': \'https://www.zeldadungeon.net/the-nintendo-switch-has-sold-125-62-million-units-breath-of-the-wild-has-sold-29-81-million-units/\', \'content\': "A Link to the Past\\nLink\'s Awakening\\nOcarina of Time\\nMajora\'s Mask\\nOracle of Seasons\\nOracle of Ages\\nThe Wind Waker\\nThe Minish Cap\\nTwilight Princess\\nPhantom Hourglass\\nSpirit Tracks\\nSkyward Sword\\nA Link Between Worlds\\nBreath of the Wild\\nTears of the Kingdom\\nAge o

In [33]:
from fastapi import FastAPI
import uvicorn
from langserve import add_routes

app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    chain,
    path="/research-assistant",
)

uvicorn.run(app, host="localhost", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop